#### Prepare notebook with relevant modules

In [1]:
!pip install -U odp-sdk --quiet

DEPRECATION: Loading egg at /srv/conda/envs/notebook/lib/python3.11/site-packages/odp-0.0.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.17.56 requires cryptography<40.0.2,>=40.0.0, but you have cryptography 44.0.1 which is incompatible.
pyopenssl 23.2.0 requires cryptography!=40.0.0,!=40.0.1,<42,>=38.0.0, but you have cryptography 44.0.1 which is incompatible.


In [2]:
from odp.client import OdpClient # The SDK

from shapely.geometry import box

import pandas as pd
import matplotlib.pyplot as plt
import pydeck as pdk

In [3]:
def create_h3_pydeck_map(df, hex_column, value_column, scale_factor=500, center_lat=58.0, center_lon=10.0, zoom=5):
    return pdk.Deck(
        layers=[pdk.Layer( 'H3HexagonLayer', df, get_hexagon=hex_column, get_fill_color=f'[254, 119, 76, {scale_factor} * {value_column}]', opacity=1, extruded=False,
            get_line_color=[32, 10, 58, 10], line_width_min_pixels=1, pickable=True, filled=True, stroked=True )],
        initial_view_state=pdk.ViewState(latitude=center_lat, longitude=center_lon, zoom=zoom, pitch=0), map_provider="mapbox", map_style='mapbox://styles/oceandatafoundation/clwg6xklg00an01pcgmeufjxq', api_keys={'mapbox': 'pk.eyJ1Ijoib2NlYW5kYXRhZm91bmRhdGlvbiIsImEiOiJjazk5bGxpNWkwYWU1M2Vya3hkcHh4czdrIn0.yf7kIiPfDNE7KP9_9wTN6A'})

In [8]:
base_url = "https://odcat.dev.hubocean.io"
client = OdpClient(base_url=base_url)

In [20]:
## Request the dataset from the catalog using the UUID:
osa_table_dataset = client.catalog.get(("7af5bc0f-c12a-451b-9834-fa6c4e0a2c87"))
osa_data = client.table_v2(osa_table_dataset)
osa_table_dataset.metadata.display_name

'Ocean Sensitive Areas - version 1.0.0 - table'

In [12]:
next(osa_data.select().dataframes())

,shannon,simpson,mangrove,hex6,seamount,cold_water_coral,seagrass,geometry,coral,S,shannon_norm,S_raw
0,0.000,1.000,0.0,86d090007ffffff,0.0,0.0,0.0,"POLYGON ((1.4824 -53.51, 1.4521 -53.5387, 1.47...",0.0,1.000,0.000,0.000
1,0.000,1.000,0.0,86d09000fffffff,0.0,0.0,0.0,"POLYGON ((1.5662 -53.5373, 1.536 -53.566, 1.56...",0.0,1.000,0.000,0.000
2,0.000,1.000,0.0,86d090017ffffff,0.0,0.0,0.0,"POLYGON ((1.395 -53.5394, 1.3647 -53.5681, 1.3...",0.0,1.000,0.000,0.000
3,0.637,0.556,0.0,86d090057ffffff,0.0,0.0,0.0,"POLYGON ((1.6501 -53.5645, 1.6199 -53.5933, 1....",0.0,2.311,0.079,0.013
4,0.000,1.000,0.0,86d09005fffffff,0.0,0.0,0.0,"POLYGON ((1.734 -53.5917, 1.7038 -53.6204, 1.7...",0.0,1.000,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.000,1.000,0.0,86dc32b37ffffff,1.0,0.0,0.0,"POLYGON ((-11.4482 -56.7962, -11.41 -56.771, -...",0.0,17.500,0.000,0.167
1996,0.000,1.000,0.0,86dc32b47ffffff,0.0,0.0,0.0,"POLYGON ((-11.8069 -56.7075, -11.7685 -56.6824...",0.0,1.000,0.000,0.000
1997,0.000,1.000,0.0,86dc32b4fffffff,1.0,0.0,0.0,"POLYGON ((-11.8925 -56.6712, -11.8541 -56.6462...",0.0,17.500,0.000,0.167
1998,0.451,0.722,0.0,86dc32b57ffffff,0.0,0.0,0.0,"POLYGON ((-11.7064 -56.688, -11.6681 -56.6629,...",0.0,1.928,0.056,0.009


In [13]:
# Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 56
lat_max = 62
lon_min = 8
lon_max = 12

In [14]:
# Enter min and max latitude and longitude values to create a bounding box polygon below. Or use the structure below to add any Well-Known-Text or GeoJSON defined polygon.
lat_min = 18
lat_max = 21
lon_min = -77
lon_max = -72

In [15]:
query_geometry = box(lon_min, lat_min, lon_max, lat_max).wkt
query_geometry

'POLYGON ((-72 18, -72 21, -77 21, -77 18, -72 18))'

In [21]:
df = pd.concat(list(osa_data.select(f'geometry within "{query_geometry}"', cols=['hex6','shannon_norm']).dataframes()), ignore_index=True)

In [22]:
df.tail(2)

,hex6,shannon_norm
1524,866736d6fffffff,0.0
1525,866736d77ffffff,0.0


In [23]:
# Vizualize data sample for the Shannon index. You can swap out shannon_norm for any other attribute e.g. seagrass
map_viz = create_h3_pydeck_map(df, 'hex6', 'simpson', scale_factor=500, center_lat=((lat_min + lat_max) / 2), center_lon=((lon_min + lon_max) / 2), zoom=6)
map_viz

{
  "initialViewState": {
    "latitude": 19.5,
    "longitude": -74.5,
    "pitch": 0,
    "zoom": 6
  },
  "layers": [
    {
      "@@type": "H3HexagonLayer",
      "data": [
        {
          "hex6": "864590537ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "86459061fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864590c87ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864590c9fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864590cafffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864590cf7ffffff",
          "shannon_norm": 0.137
        },
        {
          "hex6": "864592547ffffff",
          "shannon_norm": 0.238
        },
        {
          "hex6": "864592667ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645926b7ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864592927ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864592937ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864592d07ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864592d17ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864592d27ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864592d2fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864592d37ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864593c1fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864594487ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864594497ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "86459449fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645944b7ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864596167ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "86459616fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864596247ffffff",
          "shannon_norm": 0.137
        },
        {
          "hex6": "864596257ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "86459626fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "86459628fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "86459629fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645962c7ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645962d7ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645962e7ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645962efffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645962f7ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "86459644fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645964b7ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645964c7ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864596587ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "86459658fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864596597ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "86459659fffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645965a7ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "8645965b7ffffff",
          "shannon_norm": 0.0
        },
        {
          "hex6": "864596647ffffff",
          "shannon_norm": 0.359
        },
        {
          "hex6": "86

In [ ]:
### Select data from a single point

In [33]:
list(osa_data.select(f'geometry contains "POINT(128.38 0.39)"').rows())

[{'shannon': 0.0,
  'simpson': 0.0,
  'mangrove': 1.0,
  'hex6': '867393917ffffff',
  'seamount': 0.0,
  'cold_water_coral': 0.0,
  'seagrass': 1.0,
  'geometry': 'POLYGON ((128.349 0.4004, 128.3434 0.3648, 128.3711 0.3417, 128.4045 0.3542, 128.4101 0.3898, 128.3824 0.4129, 128.349 0.4004))',
  'coral': 1.0,
  'S': 50.5,
  'shannon_norm': 0.0,
  'S_raw': 0.5}]

In [34]:
next(osa_data.select('geometry contains "POINT(128.38 0.39)"').dataframes())

,shannon,simpson,mangrove,hex6,seamount,cold_water_coral,seagrass,geometry,coral,S,shannon_norm,S_raw
0,0.0,0.0,1.0,867393917ffffff,0.0,0.0,1.0,"POLYGON ((128.349 0.4004, 128.3434 0.3648, 128...",1.0,50.5,0.0,0.5
